### Use SageMaker Feature Store and Apache Spark to generate point-in-time queries to implement Time Travel
The following notebook uses SageMaker Feature Store and Apache Spark to build out a set of Dataframes and queries that provide a pattern for using "Time Travel" capabilities. We will demonstrate how to build a "point-in-time" feature sets by starting with raw transactional data, joining that data with records from the Offline Store, and then building an "entity" dataset to define the items we care about and the timestamp of reference. Techniques include building Spark Dataframes, using outer and inner table joins, using query filters to prune items outside our timeframe, and finally ReduceByKey to reduce the final the dataset. 

#### Install Faker library to help generate timestamps within a given range

In [1]:
!pip install Faker

distutils: /home/ec2-user/anaconda3/envs/python3/include/python3.6m/UNKNOWN
sysconfig: /home/ec2-user/anaconda3/envs/python3/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /home/ec2-user/anaconda3/envs/python3/include/python3.6m/UNKNOWN
sysconfig: /home/ec2-user/anaconda3/envs/python3/include/python3.6m
user = False
home = None
root = None
prefix = None


In [2]:
# Faker
from faker import Faker

# Import pyspark and build Spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import max as sql_max
from pyspark.sql.functions import min as sql_min
from pyspark.sql.types import StringType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType

from pyspark import SparkContext, SparkConf
import sagemaker_pyspark
import datetime

# Configure Spark to use the SageMaker Spark dependency jars
classpath = ":".join(sagemaker_pyspark.classpath_jars())


In [3]:
spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .getOrCreate())


In [4]:
sc = spark.sparkContext
print(sc.version)

2.3.4


In [5]:
SEED = 123456
faker = Faker()
faker.seed_locale('en_US', 0)
faker.seed_instance(SEED)

In [6]:
import sagemaker

BUCKET = sagemaker.Session().default_bucket()
print(BUCKET)

sagemaker-us-east-1-572539092864


In [7]:
import os

BASE_PREFIX = "sagemaker-featurestore-blog"

OFFLINE_STORE_BASE_URI = f's3://{BUCKET}/{BASE_PREFIX}'

AGG_PREFIX = os.path.join(BASE_PREFIX, 'aggregated')
print(f'S3 Aggregated Prefix: {AGG_PREFIX}')

AGG_FEATURES_PATH_S3 = f"s3://{BUCKET}/{AGG_PREFIX}/"
AGG_FEATURES_PATH_PARQUET = f"s3a://{BUCKET}/{AGG_PREFIX}/"

S3 Aggregated Prefix: sagemaker-featurestore-blog/aggregated


In [8]:
from sagemaker.s3 import S3Downloader

file_list = S3Downloader.list(AGG_FEATURES_PATH_S3)

print(f'Using S3 path: {AGG_FEATURES_PATH_S3}')
print("Found files: \n" + "\n".join(file_list))

Using S3 path: s3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-blog/aggregated/
Found files: 
s3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-blog/aggregated/_SUCCESS
s3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-blog/aggregated/part-00000-c8e0ecfd-2bff-4ceb-adb4-01368ce19237-c000.csv


#### Let's retreive our credit card transaction data

In [9]:
transactions_df = spark.read.options(Header=True).csv(AGG_FEATURES_PATH_PARQUET)

In [10]:
transactions_df.printSchema()
transactions_df.count()

root
 |-- tid: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- fraud_label: string (nullable = true)
 |-- num_trans_last_60m: string (nullable = true)
 |-- avg_amt_last_60m: string (nullable = true)
 |-- num_trans_last_7d: string (nullable = true)
 |-- avg_amt_last_7d: string (nullable = true)
 |-- amt_ratio1: string (nullable = true)
 |-- amt_ratio2: string (nullable = true)
 |-- count_ratio: string (nullable = true)



500001

In [11]:
transactions_df.show(5)

+--------------------+--------------------+----------------+------+-----------+------------------+----------------+-----------------+------------------+--------------------+--------------------+------------------+
|                 tid|          event_time|          cc_num|amount|fraud_label|num_trans_last_60m|avg_amt_last_60m|num_trans_last_7d|   avg_amt_last_7d|          amt_ratio1|          amt_ratio2|       count_ratio|
+--------------------+--------------------+----------------+------+-----------+------------------+----------------+-----------------+------------------+--------------------+--------------------+------------------+
|bea07140254b4d15b...|2021-01-01T20:44:...|4006080197832643| 674.3|          0|                 1|           674.3|                1|             674.3|                 1.0|                 1.0|               1.0|
|ecb4025cc15072ea0...|2021-01-01T20:47:...|4006080197832643| 59.33|          0|                 2|         366.815|                2|           

#### Use Sagemaker Client to retrieve info about Feature Group
We will use the `describe_feature_group` method to lookup the S3 Uri location of the Offline Store data files.

In [12]:
from sagemaker import get_execution_role
import sagemaker
import boto3

role = get_execution_role()
sm_client = boto3.Session().client(service_name='sagemaker')


In [13]:
# Identify name of the Feature Group that contains aggregated features for our transaction data
FEATURE_GROUP = 'cc-agg-batch-fg'

feature_group_info = sm_client.describe_feature_group(FeatureGroupName=FEATURE_GROUP)
feature_group_info

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/cc-agg-batch-fg',
 'FeatureGroupName': 'cc-agg-batch-fg',
 'RecordIdentifierFeatureName': 'cc_num',
 'EventTimeFeatureName': 'trans_time',
 'FeatureDefinitions': [{'FeatureName': 'tid', 'FeatureType': 'String'},
  {'FeatureName': 'cc_num', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_trans_last_7d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_amt_last_7d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'trans_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2021, 4, 30, 21, 56, 10, 133000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-blog',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-blog/572539092864/sagemaker/us-east-1/offline-store/c

In [14]:
# Lookup S3 Location of Offline Store

resolved_offline_store_s3_location = feature_group_info['OfflineStoreConfig']['S3StorageConfig']['ResolvedOutputS3Uri']

# Spark's Parquet file reader requires replacement of 's3' with 's3a'
offline_store_s3a_uri = resolved_offline_store_s3_location.replace("s3:", "s3a:")

print(offline_store_s3a_uri)

s3a://sagemaker-us-east-1-572539092864/sagemaker-featurestore-blog/572539092864/sagemaker/us-east-1/offline-store/cc-agg-batch-fg-1619819770/data


In [15]:
# Read Offline Store data
feature_store_df = spark.read.parquet(offline_store_s3a_uri)

In [16]:
feature_store_df.printSchema()
feature_store_df.count()

root
 |-- tid: string (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- num_trans_last_7d: long (nullable = true)
 |-- avg_amt_last_7d: double (nullable = true)
 |-- event_time: string (nullable = true)
 |-- trans_time: double (nullable = true)
 |-- write_time: timestamp (nullable = true)
 |-- api_invocation_time: timestamp (nullable = true)
 |-- is_deleted: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)



10001

In [17]:
feature_store_df.show(5)

+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+
|                 tid|          cc_num|num_trans_last_7d|avg_amt_last_7d|         event_time|   trans_time|          write_time|api_invocation_time|is_deleted|year|month|day|hour|
+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+
|13e125378d6ce52e7...|4006080197832643|               14|          429.3|2021-01-31 02:29:52|1.619820251E9|2021-04-30 22:09:...|2021-04-30 22:04:11|     false|2021|    4| 30|  22|
|17f1f7526762fa019...|4008569092490794|               14|         341.62|2021-01-31 02:15:13|1.619820252E9|2021-04-30 22:09:...|2021-04-30 22:04:11|     false|2021|    4| 30|  22|
|e66dfa41d179d32b7...|4015965906982664|               14|         838.93|2021-01-31 21:42:18|1.61982

#### Create an enhanced set of features by joining raw transaction data with aggregate features from the Offline Store

In [18]:
# Join the raw transactons table to the aggregate feature table 

enhanced_df = (transactions_df.join(feature_store_df, transactions_df.tid == feature_store_df.tid, "left_outer")
    .drop(transactions_df.tid)
    .drop(transactions_df.cc_num)
    .drop(transactions_df.num_trans_last_7d)
    .drop(transactions_df.avg_amt_last_7d)
    .drop(transactions_df.event_time))

In [19]:
enhanced_df.printSchema()
enhanced_df.count()

root
 |-- amount: string (nullable = true)
 |-- fraud_label: string (nullable = true)
 |-- num_trans_last_60m: string (nullable = true)
 |-- avg_amt_last_60m: string (nullable = true)
 |-- amt_ratio1: string (nullable = true)
 |-- amt_ratio2: string (nullable = true)
 |-- count_ratio: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- num_trans_last_7d: long (nullable = true)
 |-- avg_amt_last_7d: double (nullable = true)
 |-- event_time: string (nullable = true)
 |-- trans_time: double (nullable = true)
 |-- write_time: timestamp (nullable = true)
 |-- api_invocation_time: timestamp (nullable = true)
 |-- is_deleted: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)



500001

### Sample Time Travel query from Studio

##### Simplified query for datasets up to 100 GB

SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY EventTime
    ORDER BY  EventTime desc, Api_Invocation_Time DESC, write_time DESC) AS row_number
    FROM sagemaker_featurestore.identity-feature-group-03-20-32-44-1614803787
    where EventTime <= timestamp '<timestamp>')
    -- replace timestamp '<timestamp>' with just <timestamp>  if EventTimeFeature is of type fractional
WHERE row_number = 1 and
NOT is_deleted

### Build Entity Dataframe that spans the intended time window

In [20]:
import random

# Num samples in entity dataframe
NUM_RANDOM_SAMPLES = 500

cc_num_list = transactions_df.rdd.map(lambda x: x.cc_num).collect()

In [21]:
cc_num_sample = random.sample(cc_num_list, NUM_RANDOM_SAMPLES)
#print(cc_num_sample)

In [22]:
# Build list of tuples containing real credit card numbers (cc_num) with faked timestamps within our time window
start = datetime.datetime.strptime('2021-01-31 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.datetime.strptime('2021-01-31 23:00:00', '%Y-%m-%d %H:%M:%S')

samples = list()
for r in range(NUM_RANDOM_SAMPLES):
    row = []
    fake_timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime('%Y-%m-%d %H:00:00')
    row.append(cc_num_sample[r])
    row.append(fake_timestamp)
    samples.append(row)
    
#print(samples)

In [23]:
# Create and show the Entity Dataframe
# (e.g. the dataframe that defines our set of credit card numbers and timestamps for our point-in-time queries)

entity_df_schema = StructType([
    StructField('cc_num', StringType(), False),
    StructField('joindate', StringType(), False)
])

In [24]:
# Create entity data frame

entity_df = spark.createDataFrame(samples, entity_df_schema)
entity_df.show(10)

+----------------+-------------------+
|          cc_num|           joindate|
+----------------+-------------------+
|4557052810125286|2021-01-31 10:00:00|
|4705794167507670|2021-01-31 01:00:00|
|4035461743633799|2021-01-31 06:00:00|
|4133239202304420|2021-01-31 00:00:00|
|4553580746227556|2021-01-31 02:00:00|
|4623491090561556|2021-01-31 01:00:00|
|4972070993122414|2021-01-31 09:00:00|
|4774849075253886|2021-01-31 01:00:00|
|4583082767257095|2021-01-31 04:00:00|
|4104163068719676|2021-01-31 08:00:00|
+----------------+-------------------+
only showing top 10 rows



In [25]:
# Performance Improvement: 
# This first dataframe filter serves as a performance optimization to reduce the size of dataset
# We compute the overall min and max times for the initial filtering, in one pass

# entity_df used to define bounded time window
minmax_time = entity_df.agg(sql_min("joindate"), sql_max("joindate")).collect()
print(minmax_time)

[Row(min(joindate)='2021-01-31 00:00:00', max(joindate)='2021-01-31 22:00:00')]


In [26]:
min_time, max_time = minmax_time[0]["min(joindate)"], minmax_time[0]["max(joindate)"]
print(f'min_time: {min_time}')
print(f'max_time: {max_time}')

min_time: 2021-01-31 00:00:00
max_time: 2021-01-31 22:00:00


In [27]:
print("Before filter, count: " + str(enhanced_df.count()))

Before filter, count: 500001


In [28]:
%%time

# Filter out records from after query max_time and before staleness window prior to the min_time
# NOTE: This is a performance optimization; doing this prior to individual {cc_num, query_time} filtering will be faster

# Choose a "staleness" window of time before which we want to ignore records
allowed_staleness_days = 4

# Eliminate Credit Cards (entities) who do NOT have any relevant records within our time window 
# this window represents the {max_time - min_time} delta, plus our staleness window (4 days)

# Via the staleness check, we are actually removing items when event_time is MORE than 4 days before min_time
# Usage: datediff ( enddate, startdate ) - returns days

filtered = enhanced_df.filter(
    (enhanced_df.event_time <= max_time) & 
    (datediff(lit(min_time), enhanced_df.event_time) <= allowed_staleness_days)
)

CPU times: user 0 ns, sys: 1.87 ms, total: 1.87 ms
Wall time: 45.5 ms


In [29]:
filtered.printSchema()
print("After filter, count: " + str(filtered.count()))

root
 |-- amount: string (nullable = true)
 |-- fraud_label: string (nullable = true)
 |-- num_trans_last_60m: string (nullable = true)
 |-- avg_amt_last_60m: string (nullable = true)
 |-- amt_ratio1: string (nullable = true)
 |-- amt_ratio2: string (nullable = true)
 |-- count_ratio: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- num_trans_last_7d: long (nullable = true)
 |-- avg_amt_last_7d: double (nullable = true)
 |-- event_time: string (nullable = true)
 |-- trans_time: double (nullable = true)
 |-- write_time: timestamp (nullable = true)
 |-- api_invocation_time: timestamp (nullable = true)
 |-- is_deleted: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)

After filter, count: 8749


In [30]:
filtered.show(5)

+------+-----------+------------------+------------------+-------------------+--------------------+-------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+
|amount|fraud_label|num_trans_last_60m|  avg_amt_last_60m|         amt_ratio1|          amt_ratio2|        count_ratio|                 tid|          cc_num|num_trans_last_7d|avg_amt_last_7d|         event_time|   trans_time|          write_time|api_invocation_time|is_deleted|year|month|day|hour|
+------+-----------+------------------+------------------+-------------------+--------------------+-------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+
|424.77|          0|                 2|212.77499999999998| 0.4956283016513455|  0.9894397071669231|0.14285

In [31]:
filtered.select("cc_num").show(5)

+----------------+
|          cc_num|
+----------------+
|4006080197832643|
|4008569092490794|
|4014600948537520|
|4015965906982664|
|4016674905670309|
+----------------+
only showing top 5 rows



In [32]:
# Join filtered dataframe with generated entity dataframe; drop duplicate cc_num field

joined = filtered.join(entity_df, filtered.cc_num == entity_df.cc_num, "inner").drop(entity_df.cc_num)
print("Joined count: " + str(joined.count()))

Joined count: 451


In [33]:
joined.show(5)

+------+-----------+------------------+----------------+-------------------+-------------------+--------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+-------------------+
|amount|fraud_label|num_trans_last_60m|avg_amt_last_60m|         amt_ratio1|         amt_ratio2|         count_ratio|                 tid|          cc_num|num_trans_last_7d|avg_amt_last_7d|         event_time|   trans_time|          write_time|api_invocation_time|is_deleted|year|month|day|hour|           joindate|
+------+-----------+------------------+----------------+-------------------+-------------------+--------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+-------------------+
| 23.54|          0|                 1|           23

In [34]:
# Filter out data from after query time or before query time minus staleness window
# this query removes events outside the time window FOR the SPECIFIC CC (customer)
drop_future_and_stale = joined.filter(
    (joined.event_time <= entity_df.joindate)
    & (datediff(entity_df.joindate, joined.event_time) <= allowed_staleness_days)
)

print("After drop stale, count: " + str(drop_future_and_stale.count()))

After drop stale, count: 237


In [35]:
drop_future_and_stale.show(5)

+------+-----------+------------------+----------------+-------------------+-------------------+--------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+-------------------+
|amount|fraud_label|num_trans_last_60m|avg_amt_last_60m|         amt_ratio1|         amt_ratio2|         count_ratio|                 tid|          cc_num|num_trans_last_7d|avg_amt_last_7d|         event_time|   trans_time|          write_time|api_invocation_time|is_deleted|year|month|day|hour|           joindate|
+------+-----------+------------------+----------------+-------------------+-------------------+--------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+--------------------+-------------------+----------+----+-----+---+----+-------------------+
| 23.54|          0|                 1|           23

In [36]:
# Use reduceByKey to group by cc_num and keep most recent record
take_latest = (
    drop_future_and_stale.rdd.map(lambda x: (x.cc_num, x)) 
    .reduceByKey(
        lambda x, y: x if ((x.event_time) >= (y.event_time)) else y
    )  #  We could have used api_invocation_time as tie-breaker
    .values()  # drop keys
)


In [37]:
# Convert to DataFrame
latest_df = take_latest.toDF(drop_future_and_stale.schema)

In [38]:
# Drop extra columns
columns_to_drop = ["write_time", "is_deleted", "year", "month", "day", "hour", "query_time", "api_invocation_time"]
final_df = latest_df.drop(*columns_to_drop)

print('Final count: ' + str(final_df.count()))

Final count: 235


In [39]:
# Show final query results

final_df.show(10)

# To save query result to s3:
# OUTPUT_PATH = f"s3://{BUCKET}/{PREFIX}/test_query_output"
# final_df.write.parquet(OUTPUT_PATH, mode="overwrite")

+-------+-----------+------------------+----------------+--------------------+--------------------+-------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+-------------------+
| amount|fraud_label|num_trans_last_60m|avg_amt_last_60m|          amt_ratio1|          amt_ratio2|        count_ratio|                 tid|          cc_num|num_trans_last_7d|avg_amt_last_7d|         event_time|   trans_time|           joindate|
+-------+-----------+------------------+----------------+--------------------+--------------------+-------------------+--------------------+----------------+-----------------+---------------+-------------------+-------------+-------------------+
|  67.67|          0|                 1|           67.67| 0.09865522063299029| 0.09865522063299029|0.08333333333333333|c93044647cadf7d3c...|4468967287415001|               12|         685.92|2021-01-30 11:25:27| 1.61982034E9|2021-01-31 02:00:00|
|4732.31|       